# NLP ML Engineer Code Challenge Writeup

This document contains:<br>
I. challenge prompt answers <br>
II. an additional section where I explored data, techniques, results

## I. Prompt answers

### How would you evaluate your autocomplete server? If you made another version, how would you compare the two to decide which is better?

### One way to improve the autocomplete server is to give topic-specific suggestions. How would you design an auto-categorization server? It should take a list of messages and return a TopicId. (Assume that every conversation in the training set has a TopicId).

### How would you evaluate if your auto-categorization server is good?


### The autocomplete server data set we gave you is pretty small, so it doesn't take very long to load and it's fine to process on server startup.  How would you change your design if the dataset was several gigabytes?  What if it was 100 terabytes?

### What would we need to do if we had millions of agents using the Autocomplete service at the same time?

## II. Code, data, and results exploration

In [1]:
import sys
import os
import spacy
import pandas as pd
import json
from pprint import PrettyPrinter
pp = PrettyPrinter(depth=4)

In [2]:
AUTOC = os.path.abspath("{}/../../".format("challenge_writeup.ipynb"))
sys.path.insert(0, "{}/library/load_data".format(AUTOC))

In [3]:
from load_data import LoadData

### Load data

In [11]:
chats = LoadData()
chats.path
with open(chats.path) as f: 
    json_chats = json.load(f)
    pp.pprint(json_chats['Issues'])

[{'CompanyGroupId': 1,
  'IssueId': 1,
  'Messages': [{'IsFromCustomer': True,
                'Text': "Hi! I placed an order on your website and I can't "
                        'find the tracking number. Can you help me find out '
                        'where my package is?'},
               {'IsFromCustomer': True,
                'Text': 'I think I used my email address to log in.'}]},
 {'CompanyGroupId': 1,
  'IssueId': 10001,
  'Messages': [{'IsFromCustomer': True, 'Text': 'My battery exploded!'},
               {'IsFromCustomer': True,
                'Text': "It's on fire, it's melting the carpet!"},
               {'IsFromCustomer': True, 'Text': 'What should I do!'}]},
 {'CompanyGroupId': 1,
  'IssueId': 20001,
  'Messages': [{'IsFromCustomer': True,
                'Text': "I'm interested in upgrading my plan."},
               {'IsFromCustomer': True,
                'Text': 'Can you tell me a bit about Prime?'},
               {'IsFromCustomer': True,
                'T

                        'update my address and get a new cable installation.'},
               {'IsFromCustomer': False,
                'Text': 'Hey Samual, I can help you with this. Could you '
                        'please give me the details of your new address?'},
               {'IsFromCustomer': True, 'Text': '131 pioneer ct'},
               {'IsFromCustomer': True, 'Text': 'Court'},
               {'IsFromCustomer': False, 'Text': 'Zip code?'},
               {'IsFromCustomer': True, 'Text': '32801'},
               {'IsFromCustomer': False,
                'Text': 'Great let me quickly add that to the system'},
               {'IsFromCustomer': False, 'Text': 'I have added you address'},
               {'IsFromCustomer': False,
                'Text': 'I have also looked up and see we can serve that '
                        'addres'},
               {'IsFromCustomer': True, 'Text': 'Thank you'},
               {'IsFromCustomer': False,
                'Text': 'let me sched

               {'IsFromCustomer': True, 'Text': 'Hmmm, ok'},
               {'IsFromCustomer': False,
                'Text': 'Will you be around later today?'},
               {'IsFromCustomer': True, 'Text': 'Yes'},
               {'IsFromCustomer': False,
                'Text': 'Great. I will schedule a technician to come out and '
                        'look if he can resolve the problem.'},
               {'IsFromCustomer': True, 'Text': 'Great'},
               {'IsFromCustomer': False,
                'Text': 'It will be most likely between 4-5pm this afternoon'},
               {'IsFromCustomer': False, 'Text': 'is this fine?'},
               {'IsFromCustomer': True, 'Text': 'Perfect'},
               {'IsFromCustomer': False,
                'Text': 'Awesome, is there anything else I can assist you '
                        'with?'},
               {'IsFromCustomer': True, 'Text': "I think that's it. Thank you"},
               {'IsFromCustomer': False, 'Text': 'Have a gre

                'Text': 'OK my supervisory is happy to make this a once off '
                        'exception'},
               {'IsFromCustomer': True, 'Text': "I'm glad"},
               {'IsFromCustomer': False,
                'Text': 'We are willing to make the $100 refund to your credit '
                        'card.'},
               {'IsFromCustomer': True, 'Text': 'That will work'},
               {'IsFromCustomer': False,
                'Text': 'However we will need to force a password reset, '
                        'additionally you will be required to ensure you '
                        'address is updated'},
               {'IsFromCustomer': True, 'Text': 'That is okay'},
               {'IsFromCustomer': False,
                'Text': 'Great, give me one moment while I do this.'},
               {'IsFromCustomer': False,
                'Text': 'OK I have done the reset. You should receive an email '
                        'with instructions.'},
               {

               {'IsFromCustomer': False,
                'Text': 'OK Jack, I have authorized the refund.'},
               {'IsFromCustomer': True, 'Text': 'Thank you'},
               {'IsFromCustomer': False,
                'Text': 'I will however need to force a password reset, this '
                        'is company policy'},
               {'IsFromCustomer': True, 'Text': 'Ok'},
               {'IsFromCustomer': False,
                'Text': 'OK done, you should be receiving an email with '
                        'instructions and guidelines to make your account more '
                        'secure'},
               {'IsFromCustomer': True, 'Text': 'Ok thanks for your help'},
               {'IsFromCustomer': False,
                'Text': 'Please be sure to read them, to circumvent this '
                        'happing again.'},
               {'IsFromCustomer': True, 'Text': 'Will do'},
               {'IsFromCustomer': False,
                'Text': 'Can I help you wi

...

                'Text': 'which option do you think works best for you?'},
               {'IsFromCustomer': True, 'Text': 'I think the 6 is best'},
               {'IsFromCustomer': False,
                'Text': 'Great.  Please let me update your travel arrangements '
                        'and give you a confirmation number'},
               {'IsFromCustomer': True, 'Text': 'Ok'},
               {'IsFromCustomer': False,
                'Text': 'All set.  You will be flying out of Place X and '
                        'arriving at BWI at 0000, tonight.  Your confirmation '
                        'number is the following: MMMM123456'},
               {'IsFromCustomer': True, 'Text': 'Awesome thanks!'},
               {'IsFromCustomer': False,
                'Text': 'Is there anything else I can help you with?'},
               {'IsFromCustomer': True, 'Text': 'No thank you'},
               {'IsFromCustomer': False,
                'Text': 'Thank you Kevin and safe travels.'},
   

               {'IsFromCustomer': False,
                'Text': 'Hi I will help I have pulled up your flight details. '
                        'I can rebook you free of charge'},
               {'IsFromCustomer': True, 'Text': 'Great, thanks.'},
               {'IsFromCustomer': False,
                'Text': 'We have a flight leaving later today when the snow '
                        'clears at 7p. Does this work for you?'},
               {'IsFromCustomer': True,
                'Text': 'That should work. If the snow gets worse again, '
                        "what's the next flight after that?"},
               {'IsFromCustomer': False, 'Text': 'tomorrow at 8a.'},
               {'IsFromCustomer': True, 'Text': 'Okay, thanks.'},
               {'IsFromCustomer': False,
                'Text': 'ok great, I will email you all the flight details. Is '
                        'there anything else I can help you with today?'},
               {'IsFromCustomer': True, 'Text': 'Nothing 

                'Text': 'Hi I recently relocated and I need to update my '
                        'residential address.  Also I need to get someone out '
                        'here to get my cabling set up.'},
               {'IsFromCustomer': False,
                'Text': 'ok what is your account number so I can pull up your '
                        'acount?'},
               {'IsFromCustomer': True, 'Text': '356315'},
               {'IsFromCustomer': False,
                'Text': 'ok and the new address you need service to update our '
                        'accounts'},
               {'IsFromCustomer': True,
                'Text': '4972 Tyler avenue. Miami fl 33179'},
               {'IsFromCustomer': False,
                'Text': 'I have a technician available thursday at noon. will '
                        'there be someone available to receive the technican?'},
               {'IsFromCustomer': True, 'Text': "Yes I'll be here."},
               {'IsFromCustomer': Fal

                'Text': 'ok I am showing that the package was delivered but '
                        "you didn't receive it. So at this point, I can file a "
                        'claim and send a new modem out to you overnight '
                        'express it should arrive tomorrow evening.'},
               {'IsFromCustomer': True, 'Text': 'Ok that will be great'},
               {'IsFromCustomer': False,
                'Text': 'ok great thank you for being so patient and sorry for '
                        'the inconvenience this has caused you'}]},
 {'CompanyGroupId': 1,
  'IssueId': 14420001,
  'Messages': [{'IsFromCustomer': True,
                'Text': 'Hi, I bought a bookshelf online from ACME, I tried to '
                        'assemble it but it is missing a shelf. Can you send '
                        'me the missing shelf or replace this bookshelf?'},
               {'IsFromCustomer': False,
                'Text': 'hi what is the order number let me look and

               {'IsFromCustomer': False,
                'Text': 'what is the phone number on the account?'},
               {'IsFromCustomer': True,
                'Text': 'I contacted you about a missing delivery on Friday '
                        "and I haven't heard anything back yet"},
               {'IsFromCustomer': False,
                'Text': 'what is the phone number on the account?'},
               {'IsFromCustomer': True, 'Text': '654-664-7634'},
               {'IsFromCustomer': False,
                'Text': 'ok it looks like it was shipped via UPS and they '
                        'didnt get a signature so they were supposed to on all '
                        'electronics. i can file a claim and get a new one '
                        'shipped out today via fedex and should arrive by 5pm '
                        'tomorrow'},
               {'IsFromCustomer': True, 'Text': 'Ok cool. Thanks'}]},
 {'CompanyGroupId': 1,
  'IssueId': 14740001,
  'Messages': [{'IsFrom

### Inspect data

In [21]:
[[message['Text'] for message in chat['Messages'] if message['IsFromCustomer'] is False] for chat in chats.json['Issues']]

[[],
 [],
 [],
 ['Hello Werner how may I help you today?',
  'Sure I can help you with that? Could you please provide me with your new address?',
  'Let me update that information on our system',
  'OK Wernzio, I have updated your address to the system',
  'Ok let me go ahead and request a work order for a new installation. Give me a moment...',
  'OK a installation order has been places. Seems the earilest we will be able to help you  is from the 20th February onwards',
  'does that suite you?',
  'good, I have scheduled. A operator will be contacted you one day prior to lock down a time slot.',
  'Is there anything else I can help you with?',
  'Great, was my pleasure helping you. Have a great day Wernzio'],
 ['Hey Wernzio, What problems are you experiencing?',
  'Interesting, let me quickly have a look what the status for your neighborhood is.',
  'Give me a moment',
  'It seems currently we are experiencing technical problems in near your vicinity.',
  'We have sent out a technicia